In [1]:
from siphon.catalog import TDSCatalog
import intake_esm
import xarray as xr
from pathlib import Path
from IPython.display import HTML, Markdown
import panel as pn
import hvplot
import hvplot.pandas
import hvplot.xarray
from bokeh.models.tools import HoverTool
pn.extension()
import warnings
warnings.simplefilter('ignore')


In [2]:
## Climate simulations - bias adjusted
intake_path = Path('intake_cats')
cats  = list(intake_path.glob('*.json'))
# bias adjusted
c = cats[4]
cat = intake_esm.intake.open_esm_datastore(c)
options = list(cat.df['title'].unique())
options1 = [o for o in options if 'Ouranos' in o]
options1.extend( [o for o in options if 'Ouranos' not in o])
title_w = pn.widgets.Select(options=options1, width = 600)
title_w



Select(options=['Ouranos standard ensembl...], value='Ouranos standard e..., width=600)

In [5]:
@pn.depends(title_w.param.value)
def create_data_summary(dataset = title_w.param.value):
    
        
    df = cat.search(title=dataset).df
    ds = xr.open_dataset(df['path'][0], chunks=dict(time=1)).sortby(['lat','lon'])
    lic = ds.attrs['license']
    exp1 = [d.split(',') for d in df['driving_experiment'].unique()]
    exp1 = sorted(list({x for l in exp1 for x in l}))

    w = 175

    ## summary info
    thrds_access = f"https://{'/'.join([p for p in df['path'][0].split('//')[-1].split('/')[0:-1]])}/catalog.html".replace('dodsC','catalog')
    thrds_xml = thrds_access.replace('.html','.xml')
    summary = pn.Column(pn.Row(pn.pane.HTML("dataset :", width=w), pn.pane.HTML(f'<a href="{thrds_access}" target="_blank">{dataset}<a />',width=600)))
    summary.append(pn.Row(pn.pane.HTML("thredds catalog :", width=w), pn.pane.HTML(f'<a href="{thrds_xml}" target="_blank">{thrds_xml}<a />',width=600)))
    summary.append(pn.Row(pn.pane.HTML("access tutorial :", width=w), pn.pane.HTML(f'<a href="/climate_analysis.html" target="_blank">PAVICS data tutorial<a />',width=600)))
    summary.append(pn.Row(pn.pane.HTML("institution :",width=w), pn.pane.HTML(df['institution'].unique()[0], width=700)))
    summary.append(pn.Row(pn.pane.HTML("project (processing level) :", width=w), pn.pane.HTML(f"{df['project_id'].unique()[0]} ({df['processing'].unique()[0]})")))
    summary.append(pn.Row(pn.pane.HTML("frequency :", width=w), pn.pane.HTML(df['frequency'].unique()[0])))
    summary.append(pn.Row(pn.pane.HTML("variables :", width=w), pn.pane.HTML(', '.join([v for v in ds.data_vars]))))
    summary.append(pn.Row(pn.pane.HTML("driving experiment(s) :", width=w), pn.pane.HTML(', '.join(exp1))))

    out = pn.Tabs(('Summary',summary))

    ## details
    details = pn.Column(pn.Row(pn.pane.HTML('abstract : ',width=w), pn.pane.HTML(ds.attrs['abstract'],width=600)))
    for check in ['bias_adjust', 'target_data', 'target_ref']:
        for attr in [attr for attr in ds.attrs if check in attr]:
            details.append(pn.Row(pn.pane.HTML(f"{attr.replace('_',' ')}: ", width=w), pn.pane.HTML(ds.attrs[attr],width=600 )))

    details.append(pn.Row(pn.pane.HTML('more info : ', width=w), pn.pane.HTML(ds.attrs['dataset_description'], width=600)))
    out.append(('Details', details))

    ## legal
    legal = pn.Column(pn.Row(pn.pane.HTML("license type :", width=w), pn.pane.HTML(ds.attrs['license_type'], width=600)))
    legal.append(pn.Row(pn.pane.HTML("license :", width=w), pn.pane.HTML(ds.attrs['license'],width=600)))
    for attr in [attr for attr in ds.attrs if 'attribution' in attr]:
        legal.append(pn.Row(pn.pane.HTML(f"{attr.replace('_',' ')}: ", width=w), pn.pane.HTML(ds.attrs[attr],width=600 )))
    for attr in [attr for attr in ds.attrs if 'terms' in attr]:
        legal.append(pn.Row(pn.pane.HTML(f"{attr.replace('_',' ')}: ", width=w), pn.pane.HTML(ds.attrs[attr], width=600)))

    out.append(('License / Terms of use', legal))

    ## map
    v = list(ds.data_vars.keys())
    map1 = ds[v[0]].isel(time=0).hvplot.image(cmap='RdBu_r',datashade=True, hover=False, frame_height=300,frame_width=700)

    out1 = pn.Column(map1,out)
    return out1

MAX_WIDTH=850

pn.config.sizing_mode="stretch_width"

spacer = pn.Spacer(height=0, margin=0)
main_content =  pn.Column(title_w,create_data_summary,sizing_mode="stretch_width", 
    max_width=MAX_WIDTH, align="center" )

main_area = pn.Column(
    spacer, # TRICK: WONT WORK WITHOUT. YOU CAN SET HEIGHT TO 0 TO NOT TAKE UP HEIGHT
    main_content,
    sizing_mode="stretch_both",
    
)

#main_area.servable()
#from bokeh.resources import INLINE
main_area.save('Datasets_1-Climate_Simulations.html', embed=True)


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:35<01:10, 35.41s/it]

 67%|██████▋   | 2/3 [00:38<00:16, 16.24s/it]

In [6]:
pn.extension(loading_spinner='bars', loading_color='#00aa41')
main_area.servable()

Column(sizing_mode='stretch_both')
    [0] Spacer(height=0, margin=0, sizing_mode='stretch_width')
    [1] Column(align='center', max_width=850, sizing_mode='stretch_width')
        [0] Select(options=['Ouranos standard ensembl...], value='Ouranos standard e..., width=600)
        [1] ParamFunction(function, sizing_mode='stretch_width')